In [7]:
import pandas as pd


df = pd.read_excel('./data/downloaded_data/matches.xlsx')
df['Date'] = pd.to_datetime(df['Date'], format="%Y/%m/%d")

dff = df[['Date', 'home_team', 'home_elo', 'home_elo_change', 'Result']]
dff = dff.sort_values(by=['home_team', 'Date'])

dff['sum_elo'] = dff.groupby('home_team')['home_elo_change'].rolling(5, min_periods=1).sum().reset_index(level=0, drop=True)

dff['form'] = dff.groupby('home_team')['sum_elo'].shift(1)

dff.sort_values(by='Date', ascending=False, inplace=True)

tt = dff.loc[dff['home_team']=='Aston Villa']

In [2]:
hrefs_dict = {"country": [], "competition": [], "division_num": [], "hrefs": []}


pd.DataFrame(hrefs_dict)

NameError: name 'pd' is not defined

In [47]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder




class ResultTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X: pd.DataFrame, y=None):
        XX = X.pipe(self._add_result).pipe(self._transform_date)

        return XX

    def _add_result(self, df: pd.DataFrame):
        df["home_result"] = df["result"].str.partition(" :")[0].astype("int")
        df["away_result"] = df["result"].str.partition(": ")[2].astype("int")

        df = df.assign(
            result_final=lambda x: np.where(
                x["home_result"] > x["away_result"],
                1,
                np.where(x["home_result"] < x["away_result"], 2, 0),
            )
        )

        return df

    def _transform_date(self, df: pd.DataFrame):
        df["date"] = pd.to_datetime(df["date"], format="%a, %d-%b-%y").dt.strftime(
            "%Y-%m-%d"
        )

        return df


class CustomEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoder = LabelEncoder()

    def fit(self, X, y=None):
        
        combined_data = X['home_team'].append(X['away_team'])
        self.encoder.fit(combined_data)
        return self

    def transform(self, X):
        X_copy = X.copy()
        
        X_copy['home_team'] = self.encoder.transform(X['home_team'])
        X_copy['away_team'] = self.encoder.transform(X['away_team'])
        return X_copy

def transform_data(df: pd.DataFrame) -> pd.DataFrame:
    
    
    pipeline = make_pipeline(CustomEncoder())
    transformed_data = pipeline.fit_transform(df)

    return transformed_data


In [48]:
df = pd.DataFrame({
    'home_team':['Arsenal', 'Chelsea', 'Darjus'], 
    'away_team':['Man City', 'Burnley', 'Man. United']
})

transform_data(df)

C:\Users\darju\AppData\Local\Temp\ipykernel_10324\1719310186.py:52: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = X['home_team'].append(X['away_team'])


,home_team,away_team
0,0,4
1,2,1
2,3,5
